## This notebook is looking at which batters overachieved and which underachieved in 2021
#### We'll use the baseball savant expected stats compared to the actual stats so will use the baseball_savant.csv created in the savant_stats_table_maker.ipynb in utilities

### Table and Graph prep

In [1]:
import pybaseball as pb
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
from PIL import Image

In [3]:
# Show all columns
pd.set_option('display.max_columns', None)

In [4]:
# importing baseball savant csv
savant_df = pd.read_csv('/Users/charliecoleman/Desktop/Git/Repositories/baseball_analytics/csv_files/baseball_savant.csv')

In [7]:
savant_df

,Unnamed: 0,Name,ID,Year,Team,PA,POS,xwOBA_percentile,xBA_percentile,xSLG_percentile,xISO_percentile,xOBP_percentile,Barrel_percentile,Barrel%_percentile,ExitVelocity_percentile,HardHit%_percentile,K%_percentile,BB%_percentile,Whiff%_percentile,SprintSpeed_percentile,OAA_percentile,wOBA,xwOBA,xwOBA_minus_wOBA,BA,xBA,xBA_minus_BA,SLG,xSLG,xSLG_minus_SLG,K%,BB%,SwStr%,HardHit%,Barrel%,AvgExitVelo,MaxExitVelo,OAA,Spd,wRC+,WAR
0,0,Trea Turner,607208,2021,- - -,646.0,SS,85.0,98.0,80.0,55.0,81.0,69.0,40.0,54.0,77.0,74.0,15.0,64.0,100.0,82.0,0.386,0.362,0.024,0.328,0.303,0.025,0.536,0.484,0.052,0.170,0.063,0.102,0.460,7.4,89.6,112.2,3.0,6.4,142,6.9
1,1,Vladimir Guerrero Jr.,665489,2021,TOR,698.0,1B,99.0,98.0,98.0,95.0,98.0,100.0,90.0,99.0,98.0,82.0,87.0,28.0,45.0,17.0,0.419,0.416,0.003,0.311,0.306,0.005,0.601,0.591,0.010,0.158,0.123,0.124,0.552,15.1,95.1,117.4,-3.0,3.1,166,6.7
2,2,Marcus Semien,543760,2021,TOR,724.0,SS,55.0,35.0,66.0,72.0,38.0,86.0,65.0,60.0,52.0,58.0,54.0,72.0,87.0,89.0,0.368,0.331,0.037,0.265,0.244,0.021,0.538,0.453,0.085,0.202,0.091,0.087,0.413,9.8,89.7,109.5,5.0,5.3,131,6.6
3,3,Juan Soto,665742,2021,WSN,654.0,LF,100.0,97.0,92.0,85.0,100.0,90.0,84.0,94.0,94.0,90.0,100.0,80.0,52.0,86.0,0.420,0.430,-0.010,0.313,0.304,0.009,0.534,0.544,-0.010,0.142,0.222,0.063,0.524,13.3,93.0,116.6,4.0,3.5,163,6.6
4,4,Bryce Harper,547180,2021,PHI,599.0,RF,99.0,97.0,99.0,97.0,99.0,97.0,97.0,90.0,91.0,43.0,99.0,11.0,69.0,9.0,0.431,0.430,0.001,0.309,0.301,0.008,0.615,0.610,0.005,0.224,0.167,0.144,0.492,18.1,92.5,116.3,-5.0,4.3,170,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,227,Jurickson Profar,595777,2021,SDP,412.0,2B,22.0,25.0,5.0,4.0,65.0,4.0,7.0,4.0,9.0,82.0,86.0,87.0,35.0,4.0,0.292,0.302,-0.010,0.227,0.238,-0.011,0.320,0.323,-0.003,0.158,0.119,0.074,0.295,2.7,85.3,107.6,-7.0,4.9,85,-0.7
228,228,Jarred Kelenic,672284,2021,SEA,377.0,CF,31.0,8.0,41.0,62.0,18.0,36.0,65.0,19.0,38.0,11.0,61.0,35.0,54.0,22.0,0.270,0.310,-0.040,0.181,0.218,-0.037,0.350,0.408,-0.058,0.281,0.095,0.118,0.392,9.9,87.5,110.9,-2.0,4.3,73,-0.7
229,229,Jackie Bradley Jr.,598265,2021,MIL,428.0,CF,0.0,1.0,1.0,6.0,1.0,13.0,20.0,60.0,49.0,6.0,18.0,5.0,55.0,86.0,0.224,0.257,-0.033,0.163,0.198,-0.035,0.261,0.294,-0.033,0.308,0.065,0.155,0.403,5.0,89.7,108.4,4.0,6.0,35,-0.8
230,230,Cody Bellinger,641355,2021,LAD,350.0,CF,5.0,3.0,14.0,35.0,4.0,18.0,38.0,50.0,20.0,17.0,50.0,17.0,74.0,74.0,0.237,0.281,-0.044,0.165,0.208,-0.043,0.302,0.357,-0.055,0.269,0.089,0.142,0.344,7.1,89.3,107.4,2.0,5.0,48,-0.8


In [39]:

def make_scatter(dataframe, x_val:str, y_val:str, colour_val:str, hover_name= "Name", hover_data_1="Team", hover_data_2="PA"):

    fig = px.scatter(
        dataframe, x=f"{x_val}", y=f"{y_val}",
        color=colour_val,
        hover_name=hover_name,
        hover_data=[hover_data_1, hover_data_2],
        # trendline='ols',
        # trendline_color_override='purple',
        color_continuous_scale=px.colors.diverging.Tropic,
        width=1350, height=700,
        title=f'{x_val} vs {y_val}'

    )

    # add regression and standard deviation lines
    #obtain m (slope) and b(intercept) of linear regression line
    x=dataframe[f"{x_val}"]
    y=dataframe[f"{y_val}"]
    m, b = np.polyfit(x, y, 1)

    #add linear regression lines to scatterplot and
    x_linspace = np.linspace(100, x.max(), 100)
    y_sigma = y.std()
    fig.add_traces(x=x_linspace, y=m*x_linspace+b, mode='lines')
    # fig.line(x_linspace, m*x_linspace+b, label="linear regression")
    # fig.line(x_linspace, m*x_linspace+(b + y_sigma), c="k", ls="--", alpha=0.3, label="1 std dev") # one standard deviation up
    # # fig.plot(x_linspace, m*x_linspace+(b + ( 2 * y_sigma)), c="k", ls="--", alpha=0.3) # two standard deviation up
    # fig.line(x_linspace, m*x_linspace+(b + ( 3 * y_sigma)), c="k", ls=":", alpha=0.4, label="3 std dev") # three standard deviation up
    # fig.line(x_linspace, m*x_linspace+(b - y.std()), c="k", ls="--", alpha=0.3) # one standard deviation up
    


    fig.update_traces(marker_line_width=2, marker_size=10)
    fig.update_yaxes(nticks=20)
    fig.update_xaxes(nticks=40)

    # Means
    fig.add_hline(y=np.median(dataframe[y_val]), line_dash="dot", row="all", col="all",
                annotation_text=f"Mean {y_val}", 
                annotation_position="bottom right")

    fig.add_vline(x=np.median(dataframe[x_val]), line_dash="dot", row="all", col="all",
                annotation_text=f"Mean {x_val}", 
                annotation_position="bottom right")
    return fig


### First Graph: wOBA vs xwOBA

In [40]:
make_scatter(savant_df, 'wOBA', 'xwOBA', 'wRC+')

TypeError: add_traces() got an unexpected keyword argument 'x'

In [78]:
import plotly.graph_objects as go 
x = savant_df['wOBA']
y = savant_df['xwOBA']


fig = px.scatter(
    savant_df, x="wOBA", y="xwOBA",
    color='wRC+',
    hover_name='Name',
    hover_data=['Team', 'PA'],
    # trendline='ols',
    # trendline_color_override='purple',
    color_continuous_scale=px.colors.diverging.Tropic,
    width=1350, height=700,
    title=f'wOBA vs xwOBA'
    )
fig.update_traces(marker_line_width=2, marker_size=10)
fig.update_yaxes(nticks=20)
fig.update_xaxes(nticks=40)

# add regression and standard deviation lines
#obtain m (slope) and b(intercept) of linear regression line
# x=savant_df["wOBA"]
# y=savant_df["xwOBA"]
m, b = np.polyfit(x, y, 1)

#add linear regression lines to scatterplot and
x_linspace = np.linspace(0.22, x.max(), 100)
y_sigma = y.std()


# Adding regression line and std line
# Note: I know I can add regression line with trendline='ols but want to add std lines too so easier to add regression line this way too
def hex_to_rgba(h, alpha):
    '''
    converts color value in hex format to rgba format with alpha transparency
    '''
    return tuple([int(h.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)] + [alpha])

COLORS = px.colors.qualitative.D3
hex_color =COLORS[0]

fig.add_traces(go.Scatter(
    x=x_linspace, 
    y=m*x_linspace+b, 
    showlegend=False,
    name='regression',
    line=dict(
        color='rgba' + str(hex_to_rgba(h=hex_color, alpha=0.4)
    ),
    width=2,
    dash='dash')
))

fig.add_traces(go.Scatter(
    x=x_linspace, 
    y=m*x_linspace+(b+y_sigma), 
    showlegend=False,
    name='+1 st_dev',
    line=dict(
        color='rgba' + str(hex_to_rgba(h=hex_color, alpha=0.4)
    ),
    width=2,
    dash='dash')
))

fig.add_traces(go.Scatter(
    x=x_linspace, 
    y=m*x_linspace+(b-y_sigma), 
    showlegend=False,
    name='-1 st_dev',
    line=dict(
        color='rgba' + str(hex_to_rgba(h=hex_color, alpha=0.4)
    ),
    width=2,
    dash='dash')
))




fig.show()

